In [1]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split 
from sklearn.metrics import r2_score, mean_squared_error as mse
from sklearn.preprocessing import MinMaxScaler

In [2]:
df = pd.read_csv("C:/Training/Academy/Statistics (Python)/Cases/Real Estate/Housing.csv")
dum_df = pd.get_dummies(df,drop_first=True)
dum_df = dum_df.astype(float)
dum_df.head()

,price,lotsize,bedrooms,bathrms,stories,garagepl,driveway_yes,recroom_yes,fullbase_yes,gashw_yes,airco_yes,prefarea_yes
0,42000.0,5850.0,3.0,1.0,2.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
1,38500.0,4000.0,2.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,49500.0,3060.0,3.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,60500.0,6650.0,3.0,1.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
4,61000.0,6360.0,2.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [3]:
dum_df.shape

(546, 12)

In [4]:
X = dum_df.iloc[:,1:]
y = dum_df['price']
scaler = MinMaxScaler()
scalerY = MinMaxScaler()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, 
                                                    random_state=23)
y_train = scalerY.fit_transform(y_train.values.reshape(-1, 1))
X_train = scaler.fit_transform(X_train)
y_test = scalerY.transform(y_test.values.reshape(-1, 1))
X_test = scaler.transform(X_test)

In [5]:
X_torch = torch.from_numpy(X_train)
y_torch = torch.from_numpy(y_train)
print(X_torch.size())
print(y_torch.size())

torch.Size([382, 11])
torch.Size([382, 1])


In [6]:
torch.manual_seed(23)
# Create a model
model = nn.Sequential(nn.Linear(in_features=X_train.shape[1], out_features=8),
                      nn.ReLU(),
                      nn.Linear(8,6),
                      nn.ReLU(),
                      nn.Linear(6,4),
                      nn.ReLU(),
                      nn.Linear(4, 3),
                      nn.ReLU(),
                      nn.Linear(3,1),
                      nn.ReLU())

In [7]:
criterion = torch.nn.MSELoss()
# Construct the optimizer (Stochastic Gradient Descent in this case)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)

In [8]:
#model[0].weight

In [9]:
y_pred = model(X_torch.float())
#y_torch = y_torch.unsqueeze(1)
print(y_torch.shape)
print(y_pred.shape)

torch.Size([382, 1])
torch.Size([382, 1])


In [10]:
# Gradient Descent

for epoch in np.arange(0,5000):
   # Forward pass: Compute predicted y by passing x to the model
   y_pred = model(X_torch.float())

   # Compute and print loss
   loss = criterion(y_pred, y_torch.float())
   #print('epoch: ', epoch+1,' loss: ', loss.item())

   # Zero gradients, perform a backward pass, and update the weights.
   optimizer.zero_grad()

   # perform a backward pass (backpropagation)
   loss.backward()

   # Update the parameters
   optimizer.step()
   if epoch % 1000 == 0:
       print('epoch: ', epoch+1,' loss: ', loss.item())

epoch:  1  loss:  0.14013861119747162
epoch:  1001  loss:  0.006613279692828655
epoch:  2001  loss:  0.00554259167984128
epoch:  3001  loss:  0.004525571130216122
epoch:  4001  loss:  0.004155396483838558


In [11]:
#model = model.eval()
X_torch_test = torch.from_numpy(X_test)
y_pred = model(X_torch_test.float())
y_pred = y_pred.detach().numpy()
y_pred = y_pred.reshape(y_test.shape[0],)
y_pred[:5]

array([0.47556984, 0.13501006, 0.21814647, 0.10626199, 0.20742312],
      dtype=float32)

In [12]:
y_pred = y_pred.reshape(-1,1)
y_pred_orig = scalerY.inverse_transform(y_pred)
y_pred_orig[:5]

array([[103469.02 ],
       [ 47276.66 ],
       [ 60994.168],
       [ 42533.23 ],
       [ 59224.816]], dtype=float32)

In [13]:
y_test_original = scalerY.inverse_transform(y_test)
y_test_original[:5]

array([[87000.],
       [60000.],
       [65500.],
       [54000.],
       [45000.]])

### Test Set R2 Score

In [14]:
print(r2_score(y_test_original,y_pred_orig))

0.3858160334320393


### Test Set MSE

In [15]:
mse(y_test,y_pred_orig)

5513828312.627541